#  STEP 1: Install Required Packages



In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install cmake
!pip install mediapipe-model-maker
!pip install tensorflow-hub tf-keras jax protobuf==4.25.3

  Using cached ml_dtypes-0.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached wrapt-1.14.1-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
Using cached ml_dtypes-0.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
Using cached wrapt-1.14.1-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (78 kB)
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5.1:
      Successfully uninstalled ml_dtypes-0.5.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboa

# STEP 2: Import Libraries



In [1]:
from google.colab import drive
import os
import tensorflow as tf
import matplotlib.pyplot as plt

# Confirm TensorFlow version is 2.x
print("TensorFlow version:", tf.__version__)
assert tf.__version__.startswith('2')

# Mediapipe Model Maker
from mediapipe_model_maker import gesture_recognizer


TensorFlow version: 2.15.1


/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# STEP 3 : Connect to Google Drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# STEP 4: Define Dataset Path

In [3]:
dataset_path = '/content/drive/MyDrive/sign_language_dataset'

# Verify path
if os.path.exists(dataset_path):
    print(f"✅ Dataset folder found: {dataset_path}")
else:
    print(f"❌ Dataset folder not found. Please check the path: {dataset_path}")


✅ Dataset folder found: /content/drive/MyDrive/sign_language_dataset


#  STEP 5: Display Dataset Labels (Classes)

In [4]:
labels = []
for label_name in os.listdir(dataset_path):
    label_folder = os.path.join(dataset_path, label_name)
    if os.path.isdir(label_folder):
        labels.append(label_name)

print("\nClasses Detected:")
for label in labels:
    print(f" - {label}")
print(f"\nTotal classes: {len(labels)}")



Classes Detected:
 - E
 - K
 - L
 - B
 - W
 - Q
 - P
 - X
 - Y
 - V
 - C
 - Yes
 - Tell
 - J
 - M
 - Hello
 - Name
 - ILoveYou
 - none
 - D
 - Bye
 - Me
 - F
 - T
 - Deaf
 - H
 - S
 - Pen
 - A
 - O
 - I
 - R
 - G
 - Meet
 - Learn
 - No
 - Thankyou
 - N
 - Ok
 - U
 - NotOk
 - Please

Total classes: 42


# STEP 7: Visualize Sample Images from Each Class

In [5]:
NUM_IMAGES = 5  # Number of samples to display per class

for label in labels:
    label_dir = os.path.join(dataset_path, label)
    sample_images = os.listdir(label_dir)[:NUM_IMAGES]

    fig, axes = plt.subplots(1, NUM_IMAGES, figsize=(15, 3))
    fig.suptitle(f"Examples of '{label}'", fontsize=16)

    for idx, image_name in enumerate(sample_images):
        img_path = os.path.join(label_dir, image_name)
        img = plt.imread(img_path)
        axes[idx].imshow(img)
        axes[idx].axis('off')

    plt.show()

<ipython-input-5-0e5edb007b68>:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(1, NUM_IMAGES, figsize=(15, 3))


# STEP 8: Load Dataset and Split

In [7]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

# Split the dataset (80% Train, 10% Validation, 10% Test)
train_data, remaining_data = data.split(0.8)
validation_data, test_data = remaining_data.split(0.5)

print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(validation_data)}")
print(f"Test samples: {len(test_data)}")


Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Train samples: 1631
Validation samples: 204
Test samples: 204


# STEP 9: Train Gesture Recognition Model

In [8]:
hparams = gesture_recognizer.HParams(
    export_dir='exported_model',  # Where the model will be saved
    epochs=30,                    # Customize number of epochs if needed
    batch_size=16                 # You can increase batch size for GPU
)

options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)

# Create and train the model
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 42)                5418      
 out (Dense)                                                     
                                                             

#  STEP 10: Evaluate the Model on Test Data

In [9]:
loss, accuracy = model.evaluate(test_data, batch_size=1)
print(f"\n✅ Test Loss: {loss:.4f}")
print(f"✅ Test Accuracy: {accuracy:.4f}")

204/204 [==============================] - 1s 2ms/step - loss: 0.1229 - categorical_accuracy: 0.9069

✅ Test Loss: 0.1229
✅ Test Accuracy: 0.9069


# STEP 11: Export the Trained Model

In [27]:
model.export_model()







Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/canned_gesture_classifier.tflite


# STEP 12: List Exported Files

In [28]:
!ls exported_model

best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


In [29]:
!ls -la /content/
!ls -la /content/exported_model/



total 24
drwxr-xr-x 1 root root 4096 Mar 22 06:37 .
drwxr-xr-x 1 root root 4096 Mar 22 05:33 ..
drwxr-xr-x 4 root root 4096 Mar 20 13:31 .config
drwx------ 6 root root 4096 Mar 22 06:19 drive
drwxr-xr-x 4 root root 4096 Mar 22 06:41 exported_model
drwxr-xr-x 1 root root 4096 Mar 20 13:31 sample_data
total 8384
drwxr-xr-x 4 root root    4096 Mar 22 06:41 .
drwxr-xr-x 1 root root    4096 Mar 22 06:37 ..
-rw-r--r-- 1 root root   71516 Mar 22 06:38 best_model_weights.data-00000-of-00001
-rw-r--r-- 1 root root    1096 Mar 22 06:38 best_model_weights.index
-rw-r--r-- 1 root root      93 Mar 22 06:38 checkpoint
drwxr-xr-x 2 root root    4096 Mar 22 06:38 epoch_models
-rw-r--r-- 1 root root 8480581 Mar 22 06:55 gesture_recognizer.task
drwxr-xr-x 4 root root    4096 Mar 22 06:37 logs
-rw-r--r-- 1 root root    1410 Mar 22 06:55 metadata.json


In [30]:
from google.colab import files
files.download('/content/exported_model/gesture_recognizer.task')




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# STEP 13: Install Additional Packages for API

In [18]:
!pip install flask flask-cors pyngrok
